** Algorithm Description **

**Example: Filtering Mobile Phone Spam**

***Step 1*** Exploring and Preparing the Data

The first step towards constructing our classifier involves processing the raw data for
analysis. Text data are challenging to prepare, because it is necessary to transform the
words and sentences into a form that a computer can understand. We will transform
our data into a representation known as bag-of-words, which ignores word order and
simply provides a variable indicating whether the word appears at all. 

Data are downnloaded from [Machine-Learning-with-R-dataset](https://github.com/stedy/Machine-Learning-with-R-datasets), a complete description can be found at [SMS Spam Collection](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/).


In [1]:
# Definition of Working Folder

Dirname <- "/working/01_Machine_Learning/01_Appunti"
Dirs <- list.dirs(path=file.path("~"),recursive=T)
dir_wd <- names(unlist(sapply(Dirs,grep,pattern=Dirname))[1])
dir_wd <- paste(dir_wd,'Dati',sep='/')
setwd(dir_wd)
cat("Current working folder is ' : ", getwd())

Current working folder is ' :  C:/Users/T004314/Documents/GitHub/working/01_Machine_Learning/01_Appunti/Dati

**Example: Document Cluster**

In [2]:
sms_raw <- read.csv("sms_spam.csv", stringsAsFactors = FALSE)